In [1]:
import os
import random

import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline

In [15]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained('youscan/ukr-roberta-base')
    model = AutoModel.from_pretrained('youscan/ukr-roberta-base')
    return tokenizer, model

tokenizer, model = load_model()

In [9]:
def model_embedding(text, tokenizer, model):
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
    outputs = model(input_ids, output_hidden_states=True)
    emb = outputs[2]
    emb = np.array([i[0].detach().numpy() for i in emb])[:]
    emb = emb.mean(axis=(0, 1))
    return emb

In [6]:
def load_data():
    data = pd.read_csv('../resources/data/data.csv', index_col=0)
    data['descr'] = data['descr'].str.lower()
    return data

def train_test_split(data):
    train_data = data[data['user'] <= 4].copy()
    test_data = data[data['user'] > 4].copy()
    return train_data, test_data

data = load_data()
train_data, test_data = train_test_split(data)

In [8]:
labels_list = train_data['word'].values
descriptions_list = train_data['descr'].values
embeddings_list = {}

In [10]:
for label, descr in zip(labels_list, descriptions_list):
    tmp = embeddings_list.get(label, [])
    tmp.append(model_embedding(descr, tokenizer, model))
    embeddings_list[label] = tmp

In [11]:
for label, embeddings in embeddings_list.items():
    embeddings_list[label] = np.mean(embeddings, axis=0)

In [12]:
test_labels_list = test_data['word'].values
test_descriptions_list = test_data['descr'].values

In [13]:
def distance(a, b):
    return sum([(i - j) ** 2 for i, j in zip(a, b)]) ** .5

In [14]:
t = 0
pred_labels = []
for ind in range(len(test_descriptions_list)):

    label = test_labels_list[ind]
    descr = test_descriptions_list[ind]
    print(f'LABEL {label}\tDESCR: {descr}')

    test_emb = model_embedding(descr, tokenizer, model)

    scores = list((distance(el, test_emb), k) for k, el in embeddings_list.items())
    sorted_scores = sorted(scores, key=lambda x: x[0])
    best_preds = sorted_scores[0][1]
    print(label, best_preds)
    pred_labels.append(best_preds)
    if label == best_preds:
        t += 1
        
print(t / len(test_descriptions_list))

LABEL кінь	DESCR: тварина, яку запрягають у віз, її силу прирівнюють до одиниць вимірювання міцності автомобілів.
кінь кінь
LABEL зебра	DESCR: тварина, яка має гриву, але не кінь.
зебра пінгвін
LABEL корова	DESCR: велика, рогата худоба.
корова олень
LABEL ведмідь	DESCR: впадає в сплячку взимку, любить мед.
ведмідь морквина
LABEL олень	DESCR: цим звіром нарікають людей, які чогось не розуміють.
олень пінгвін
LABEL страус	DESCR: птах, який ховає голову в пісок.
страус страус
LABEL курка	DESCR: домашня птиця, літає погано і недалеко.
курка курка
LABEL пінгвін	DESCR: птах, який живе на північному полюсі, темно синього кольору з білим животом.
пінгвін зебра
LABEL сова	DESCR: птах, полює на дрібних гризунів, переважно вночі.
сова сова
LABEL лебідь	DESCR: красивий, білий птах, переважно перебуває у воді.
лебідь сова
LABEL морквина	DESCR: овоч, який дуже полюбляють зайці, конусної форми.
морквина морквина
LABEL яблуко	DESCR: кислий, солодкий із кісточками всередині, корисно їсти зі шкіркою.
яб

In [17]:
import joblib

In [22]:
joblib.dump(embeddings_list, '../resources/test.joblib')

['../resources/test.joblib']